# Web-scraping: сбор данных из баз данных и интернет-источников
*Алла Тамбовцева, НИУ ВШЭ*

## Практикум 7.1. Управление браузером с Selenium: скачивание файлов

### Поиск с помощью запросов XPATH

Немного теории.

**XML** (от *eXtended MarkUp Language*) – язык разметки, только в отличие от HTML не позволяет регулировать внешний вид страницы, а просто хранит данные в виде строки с удобными тэгами.

На XML-файл можно смотреть как на хранилище, откуда по запросу динамически подгружаются данные для подстановки в HTML-файл. Смысл: когда нам нужно постоянно обновлять информацию на веб-странице (каталоги товаров в магазине, данные о погоде, курсе валют), не нужно каждый раз переписывать HTML-файл, достаточно изменить XML-файл, а из него уже информация «подтянется» на страницу с помощью запроса, написанного на JavaScript.

**XPATH** (от *XML Path Language*) – язык запросов в XML-файлу, который можно использовать и для HTML тоже.

Примеры запросов (честно взяты [отсюда](https://www.w3schools.com/xml/xpath_intro.asp), очень полезный тьюториал по XPATH, у них же есть классные материалы по XML):

* `//title[@lang]`: все элементы с тэгом `<title>`, имеющие атрибут `lang`;
* `//title[@lang='en']` : все элементы с тэгом `<title>`, имеющие атрибут `lang`, равный `'en'`;
* `//title[@*]`: все элементы с тэгом `<title>`, имеющие хоть какие-нибудь атрибуты.

Запросы на XPATH поддерживаются Selenium (внутри коллекции `By` есть вариант `XPATH`), а значит, эти запросы можно использовать в случае, если обычный поиск по тэгу/классу/тексту кажется неудобным.

### Часть 1: скачиваем PDF и ZIP

Импортируем необходимые модули и коллекции методов:

In [ ]:
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

Начинаем новую сессию работы с браузером – новое окно браузера:

In [ ]:
br = wd.Chrome()

Переходим на сайт для поиска нот (*why not?*):

In [ ]:
br.get("https://ponotam.ru/")

Давайте для практики будем искать все необходимые элементы (поля для поиска, ссылки, кнопки) через XPATH.
Сначала нам нужно найти поле для поиска. Давайте найдем его и отправим туда значение *Happy New Year*.

In [ ]:
### YOUR CODE HERE ###

Теперь попробуем найти кнопку для активации поиска:

In [ ]:
### YOUR CODE HERE ###

Перешли на страницу с результатами, давайте заберем ссылку на единственный результат по частичному тексту ссылки и кликнем на нее.

In [ ]:
link = br.find_element(By.PARTIAL_LINK_TEXT, "Happy new Year")
link.click()

Ура! Чтобы скачать pdf-файл с нотами, нужно кликнуть на соответствующую иконку. Давайте найдем ее через XPATH и кликнем.

In [ ]:
### YOUR CODE HERE ###

Идеально! В браузере открылся pdf-файл, но как его автоматически скачать, неясно. Давайте откроем браузер с дополнительными опциями, чтобы PDF скачивался сразу, а не просто открывался в браузере.

In [ ]:
options = wd.ChromeOptions()

In [ ]:
# словарь 
# default_directory: папка для загрузок, поменяйте или закомментируйте, 
# если папка по умолчанию устраивает
# prompt_for_download: автоматическое скачивание без всплывающих вопросов
# plugins.always_open_pdf_externally: скачивание без открытия в браузере

options.add_experimental_option('prefs', {
"download.default_directory": "/Users/allat/Downloads/", 
"download.prompt_for_download": False, 
"download.directory_upgrade": True,
"plugins.always_open_pdf_externally": True
})

Открываем браузер с дополнением `options`:

In [ ]:
br = wd.Chrome(options=options)

Осталось повторить все проделанные операции по скачиванию!

In [ ]:
br.get("https://ponotam.ru/")

In [ ]:
### YOUR CODE HERE ###

In [ ]:
### YOUR CODE HERE ###

Выполните то же самое для файла с zip-архивом.

In [ ]:
### YOUR CODE HERE ###

### Часть 2: скачиваем файлы DOC

Давайте зайдем на страницу сайта Вышки, на которой хранятся бланки заявлений для приема на работу и скачаем все бланки в формате `.doc`. В данном случае задача не очень сложная: если посмотреть на исходный код страницы, можно заметить, что для скачивания файлов необходимо просто кликнуть на ссылку, которая заканчивается расширением `.doc`. Поэтому пока не будем использовать Selenium, а вытащим из исходного кода страницы подходящие ссылки:

In [ ]:
import requests
from bs4 import BeautifulSoup
from time import sleep

In [ ]:
page = requests.get("https://hr.hse.ru/blanki")
soup = BeautifulSoup(page.text)

In [ ]:
L = soup.find_all("a")
docs = []

### YOUR CODE HERE ###

In [ ]:
# не все ссылки на doc полные 
# для экономии времени отфильтруем только полные

docs_full = []
for doc in docs:
    if doc.startswith("http"):
        docs_full.append(doc)

Теперь осталось только прокликать все ссылки из списка с помощью Selenium, по умолчанию документы сохранятся в папку с загрузками. На всякий случай добавим задержку в 5 секунд после каждого скачивания:

In [ ]:
br = wd.Chrome()
for i in docs_full:
    br.get(i)
    sleep(5)